# Word2Vec con Gensim


## Word2Vec
[Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec) es un modelo de procesamiento de lenguaje natural (NLP) desarrollado por Google en 2013 que se utiliza para representar palabras como vectores numéricos en un espacio vectorial de alta dimensionalidad. La idea principal detrás de Word2Vec es que las palabras que aparecen en contextos similares tienden a tener significados similares. Por lo tanto, al representar palabras como vectores en un espacio vectorial, las palabras con significados similares estarán cercanas entre sí en ese espacio.

Word2Vec se basa en una arquitectura de redes neuronales artificiales y se entrena en grandes cantidades de texto sin etiquetar para aprender las representaciones vectoriales de palabras. Hay dos modelos principales de Word2Vec:

1. **CBOW (Continuous Bag of Words)**: En este modelo, se intenta predecir una palabra objetivo basada en un contexto circundante de palabras de entrada. Por ejemplo, dadas las palabras "el gato está en el tejado", CBOW podría predecir "gato" dados los contextos "el", "está", "en", "el", "tejado".

2. **Skip-gram**: En este modelo, se invierte el enfoque de CBOW. En lugar de predecir una palabra objetivo a partir de su contexto, el modelo Skip-gram intenta predecir el contexto (las palabras circundantes) a partir de una palabra objetivo. En el ejemplo anterior, dado "gato", el modelo Skip-gram podría intentar predecir "el", "está", "en", "el", "tejado".

Los vectores de palabras resultantes son llamados [**"word embeddings"**](https://www.tensorflow.org/text/guide/word_embeddings) y son representaciones numéricas densas de las palabras. Estos vectores capturan relaciones semánticas y similitudes entre palabras. Por ejemplo, si tienes vectores de palabras entrenados con Word2Vec, puedes realizar operaciones vectoriales como "vec(rey) - vec(hombre) + vec(mujer)" y el resultado estaría cerca del vector de la palabra "reina", lo que demuestra la capacidad de Word2Vec para capturar relaciones de género entre palabras.

Word2Vec ha demostrado ser una técnica poderosa en NLP y se utiliza en una amplia variedad de aplicaciones, como la recuperación de información, la traducción automática, el análisis de sentimientos y la recomendación de contenido. Estas representaciones vectoriales de palabras son una parte fundamental de muchos modelos de NLP más avanzados.

##Gensim

[Gensim](https://radimrehurek.com/gensim/) es una biblioteca de procesamiento de lenguaje natural (NLP) en Python que se utiliza principalmente para construir, entrenar y utilizar modelos de representación de texto, como Word2Vec y otros algoritmos de incrustación de palabras. Es una herramienta versátil y eficiente para trabajar con texto y crear vectores de palabras (word embeddings) a partir de grandes colecciones de texto. Gensim es ampliamente utilizado en aplicaciones de minería de texto, análisis de texto y procesamiento de lenguaje natural.


## Instalación de librerías y carga de dataset

La biblioteca ***datasets*** es una herramienta de Python desarrollada por [***Hugging Face***](https://huggingface.co/) que proporciona acceso fácil y rápido a una amplia variedad de conjuntos de datos utilizados comúnmente en el procesamiento de lenguaje natural (NLP) y tareas de aprendizaje automático. Estos conjuntos de datos incluyen corpus de texto, datos de entrenamiento y prueba para tareas específicas de NLP.

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:00:00


Importamos la bibliotecas necesarias para realizar el entrnamiento.

In [2]:
from gensim.models import Word2Vec
import pandas as pd # Para trabajar el dataset
import re # Expresiones regulares
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords #NLTK es una biblioteca para NLP
from nltk.tokenize import word_tokenize

Cargaremos un gran dataset de mas de 15.000.000 de ejmplos de frases. Este proceso puede tardar aproximadamente unos 10 minutos dependiendo de la Conexion a Internet.

In [3]:
from datasets import load_dataset # Cargamos el Dataset de un corpus en Español

dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl")

Generating train split:   0%|          | 0/15510649 [00:00<?, ? examples/s]

In [4]:
dataset_corpus # Revisamos el formato del Dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15510649
    })
})

Como es un corpus muy grande solo vamos a tomar una porción en esta ocacion para entrenar.

In [5]:
subset = dataset_corpus['train'].select(range(1000000)) # Solo tomamos 1 millon

In [6]:
subset[0:2]

{'text': ['lavado de cerebro a través de los medios de comunicación, y amenaza de fuerza a través de los militares.',
  'Sin un constante aluvión de doble cañón, requiriendo la complicidad de los seres humanos para reprimir y engañar a sus semejantes, su tan cacareada magia rápidamente se desvanecería y se disiparía.']}

En el NLP, las ***"stopwords"*** (también conocidas como palabras vacías o palabras de parada) son palabras que generalmente se filtran o se eliminan del texto durante el preprocesamiento antes de realizar análisis de texto o tareas de procesamiento de lenguaje. Estas palabras son consideradas como palabras comunes y frecuentes en un idioma determinado, pero que generalmente no aportan un significado sustancial al texto y, por lo tanto, se pueden omitir sin perder información esencial.

Ejemplos de stopwords en inglés incluyen palabras como "a", "an", "the", "in", "on", "is", "and", "of", "it", "to", entre otras. En español, ejemplos de stopwords incluyen palabras como "el", "la", "de", "en", "un", "una", "y", "es", "para", "con", "por", "lo", entre otras.

El filtrado o eliminación de stopwords es una práctica común en NLP porque ayuda a reducir la dimensionalidad de los datos, mejorar la eficiencia computacional y eliminar ruido en el análisis de texto. Sin embargo, es importante tener en cuenta que la lista de stopwords puede variar según el idioma y el contexto de la tarea de procesamiento de texto. En algunos casos, ciertas stopwords pueden ser relevantes y no deben ser eliminadas.

In [7]:
import nltk # Vamos a usar nltk para traernos las stopwords
nltk.download('stopwords')
nltk.download('punkt') # para optimizar el tokenizador

from nltk.tokenize import word_tokenize #tokenizador

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Pre-procesamiento de texto

In [8]:
# creamos el set de StopWords por fuera de la función, como constante global
# para mejorar el rendimiento
STOP_WORDS = set(stopwords.words('spanish'))
# idem para las regular expressions que luego reusaremos mil veces
URLS_RE = re.compile(r'(https?|www)\S+', flags=re.MULTILINE)
SOCIAL_RE = re.compile(r'[@#]\w+')


def clean_text(sentence_batch):
     # extrae el texto de la entrada
    text_list = sentence_batch['text']

    cleaned_text_list = []
    for text in text_list:
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina URLs (usando la regex precompilada)
        text = URLS_RE.sub('', text)

        # Elimina las menciones @ y '#' de las redes sociales
        text = SOCIAL_RE.sub('', text)

        # Elimina los caracteres de puntuación
        text = strip_punctuation(text)

        # Elimina los números
        text = strip_numeric(text)

        # Elimina las palabras cortas
        text = strip_short(text,minsize=2)

        # Elimina las palabras comunes (stop words)
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in STOP_WORDS]

        cleaned_text_list.append(filtered_text)


    # Devuelve el texto limpio
    return {'text': cleaned_text_list}

In [9]:
sentences_corpus = subset.map(clean_text, batched=True) # Aplicamos la funcion for lotes

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [10]:
sentences_corpus['text'][:3]

[['lavado',
  'cerebro',
  'través',
  'medios',
  'comunicación',
  'amenaza',
  'fuerza',
  'través',
  'militares'],
 ['constante',
  'aluvión',
  'doble',
  'cañón',
  'requiriendo',
  'complicidad',
  'seres',
  'humanos',
  'reprimir',
  'engañar',
  'semejantes',
  'tan',
  'cacareada',
  'magia',
  'rápidamente',
  'desvanecería',
  'disiparía'],
 ['realidad',
  'nuevo',
  'om',
  'sólo',
  'puede',
  'mantener',
  'ilusión',
  'supremacía',
  'mágica',
  'siempre',
  'reprima',
  'desvíe',
  'potencial',
  'humano',
  'mora',
  'verdadera',
  'magia',
  'decir',
  'capacidad',
  'innata',
  'especie',
  'magia',
  'interactiva',
  'poderes',
  'animación',
  'diosa',
  'planetaria']]

## Carga y uso de modelo de embeddings Word2Vec

Vamos a entrenar el modelo Word2Vec con nuestro corpus limpio. Indicando que tendra 100 dimensiones, con una ventana de 5 palabras,que solo tome palabras con mas de 2 caracteres, en 6 hilos de ejecución, con skip-gram. Este proceso va a tardar un tiempo pero podemos disminuirlo (5 min aprox) si utilizamos la GPU en la colab.

In [11]:
model = Word2Vec(sentences_corpus['text'], vector_size=100, window=5, min_count= 2, workers=6, sg=1)

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

In [12]:
model.wv['rey'] # revisemos que formato tiene el embbedig de rey

array([-0.21335639, -0.13891158, -0.2570373 ,  0.27906305,  0.5452617 ,
       -0.4789913 , -0.45937023,  0.67118603, -0.4053895 , -0.33690915,
       -0.15490237, -0.5972233 ,  0.26208422,  0.4374638 , -0.04784542,
       -0.24333322, -0.13416915, -0.63014895, -0.41182578,  0.33464706,
        0.31762362,  0.46152022,  0.9133998 , -0.3878762 ,  0.54379123,
        0.5483707 , -0.6393917 , -0.9610274 , -0.34332106,  0.00826993,
       -0.6049151 ,  0.74160206, -0.0343904 , -0.5017614 , -0.68805915,
        0.6792365 , -0.6900169 , -0.19046925, -0.18153535, -0.56867045,
        0.05204853, -0.87307644,  0.00671611,  0.23553532,  0.14331008,
        0.02137338,  0.01507648,  0.73750937,  0.1806455 ,  0.36293474,
       -0.2549132 , -0.7479153 ,  0.48459595, -0.22985889,  0.01547895,
       -0.11484071,  0.02190346, -0.92841184, -0.15333915, -0.03778388,
        0.07717085, -0.05156028,  0.1344793 ,  0.00329109,  0.12036753,
        0.50104773, -0.46008208, -0.2802    , -0.6119025 , -0.09

In [15]:
## busquemos las 3 palabras mas similares a comida, ser, reina, radio
model.wv.most_similar(['comida'],topn=3)

[('comer', 0.7658618092536926),
 ('comidas', 0.7395755052566528),
 ('sabrosa', 0.7143890261650085)]

In [16]:
word_vectors = model.wv # vamos a guardar la matriz de embbedings
vectors = word_vectors.vectors # y la separamos en vectores
words = word_vectors.index_to_key # y palabras

## Almacenamiento de embeddings

Ahora vamos a generar los archivos para poder verlos en el proyector de Word2vec.

Visualizador de Embeddings:
https://projector.tensorflow.org/

In [17]:
df_vectors = pd.DataFrame(vectors)
df_vectors.to_csv('embeddings.tsv',sep='\t', index=False)

In [18]:
df_words = pd.DataFrame(words)
df_words.to_csv('labels.tsv',sep='\t', index=False)